In [1]:
PARAMS_COMMON={
    'project_from':'HYBRID-Imbalanced',
    'project_to':'RefinedTreshold-Hybrid-Test'
}

In [2]:
import sys
sys.path.append('/home/ubuntu/intelliscope/modules')
sys.path.append('../..')


#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/'+PARAMS_COMMON['project_from']]
projects=project.get_experiments()



def get_results(model=None, X_pcg=None, X_ecg=None, Y=None):    
    y_pred = model.predict([X_pcg,X_ecg], batch_size=32, verbose=1)
    y = np.argmax(Y,axis=1)
    
    return y, y_pred
    


def neptune_log_metrics(y_test=None, y_pred=None, threshold = 0.5):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)
    

    

In [3]:
project_name=PARAMS_COMMON['project_to']
X_val,Y_val, names_val = instead_data_loaders(projects[0].get_parameters(),'val',True)
X_test,Y_test, names_test = instead_data_loaders(projects[0].get_parameters(),'test', True)

Total  340
97 Images loaded across 2 Categories. Ignored blanks  8
243 Images loaded across 2 Categories. Ignored blanks  0
332 Total After removing blanks
Total  707
199 Images loaded across 2 Categories. Ignored blanks  60
508 Images loaded across 2 Categories. Ignored blanks  9
638 Total After removing blanks


In [4]:
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax



def get_optimal_thres(model, y_val, y_val_pred):
    y_val_pred_1=y_val_pred[:,1]
    fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_1)
    gmeans = sqrt(tpr * (1-fpr))
    ix = argmax(gmeans)
    
    print(f'\nfpr : {fpr[ix]} , tpr : {tpr[ix]} , gmeans : {gmeans[ix]}')
    return thresholds[ix]
    


def get_individual_metrics(y_test, y_pred, names):
    patient_preds={}
    for i in range(len(names)): # create dict: patient_record_name: [start_point, label, [pred0, pred1]] 
        name=names[i]
        record_name,record_start_point,_=name.strip().split('_')
        if record_name in patient_preds:
            patient_preds[record_name].append([int(record_start_point),y_test[i], list(y_pred[i])])
        else:
            patient_preds[record_name]=[[int(record_start_point),y_test[i], list(y_pred[i])]]

    y_test_bulk, y_pred_bulk=[],[]    
    for key in patient_preds: # return :: patient-wise y_test, y_preds
        patient_pred=patient_preds[key]
        label, pred_label=get_single_patient_pred(patient_pred, mean)

        y_test_bulk.append(label)
        y_pred_bulk.append(pred_label)

    y_test_bulk=np.array(y_test_bulk)
    y_pred_bulk=np.array(y_pred_bulk)
    
    return y_test_bulk, y_pred_bulk

def mean(list_):
    metric=np.mean(list_)
    return [1-metric,metric]

def get_single_patient_pred(patient_pred, func):
    #print(patient_pred)
    patient_pred.sort()
    
    y=np.array(patient_pred)[:,1]
    
    assert y.sum()%(y.shape[0])==0
    label=y[0] # label of the patient
    
    rows=[]
    for row in sorted(patient_pred):
        rows.append(list(row[2]))
    rows=np.array(rows)

    y_preds=np.argmax(rows, axis=1)
    pred_label=func(y_preds)
    return label, pred_label


In [5]:
X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

X_val_pcg = X_val[:,:,:,0:3]/255.0
X_val_ecg = X_val[:,:,:,3:6]/255.0

for i in range(len(projects)):
        
    project    = projects[i]
    print('\n\n',str(project))
    
    model_name = str(project)+'.h5'
    PARAMS     = project.get_parameters()
    PARAMS['tags']=PARAMS['tags'][2:-2].strip().split("', '")+ ['patientwise']
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    
    y_val, y_val_pred=get_results(model, X_val_pcg, X_val_ecg, Y_val)
    y_val, y_val_pred=get_individual_metrics(y_val, y_val_pred, names_val)
    
    optimal_thres=get_optimal_thres(model, y_val, y_val_pred)
    
    print('optimal_threshold : ',  optimal_thres, '\n')
    y_test, y_test_pred=get_results(model, X_test_pcg, X_test_ecg, Y_test)
    y_test, y_test_pred=get_individual_metrics(y_test, y_test_pred, names_test)
    
    
    sk_metrics=custom_metrics._class_metrics(y_test, y_test_pred[:, 1] > optimal_thres)
    print(sk_metrics)
    gmean=sqrt(sk_metrics['true_negative_rate']*sk_metrics['true_positive_rate'])
    PARAMS['G-Mean']=gmean
    PARAMS['optimal_threshold']=optimal_thres

    neptune.init('intelliscope/'+PARAMS_COMMON['project_to'],NEPTUNE_API_TOKEN)
    exp = neptune.create_experiment(name=str(project),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    neptune_log_metrics(y_test, y_test_pred, optimal_thres)
    neptune.stop()



 Experiment(HYBIM-1)
332/332 [==============================] - 3s 8ms/sample

fpr : 0.0 , tpr : 0.8275862068965517 , gmeans : 0.909717652294684
optimal_threshold :  0.9 

638/638 [==============================] - 5s 7ms/sample
{'accuracy': 0.7808219178082192, 'precision': 0.9555555555555556, 'recall': 0.7543859649122807, 'f1_score': 0.8431372549019609, 'f2_score': 0.7875457875457875, 'matthews_corrcoef': 0.5354626718124088, 'cohen_kappa': 0.49568221070811747, 'true_positive_rate': 0.7543859649122807, 'true_negative_rate': 0.875, 'positive_predictive_value': 0.9555555555555556, 'negative_predictive_value': 0.5, 'false_positive_rate': 0.125, 'false_negative_rate': 0.24561403508771928, 'false_discovery_rate': 0.044444444444444446}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-34


 Experiment(HYBIM-2)
332/332 [==============================] - 1s 5ms/sample

fpr : 0.0 , tpr : 0.8275862068965517 , gmeans : 0.909717652294684
optimal_threshold :  0.7777777777777778 

638/638 [==============================] - 3s 4ms/sample
{'accuracy': 0.821917808219178, 'precision': 0.9230769230769231, 'recall': 0.8421052631578947, 'f1_score': 0.8807339449541285, 'f2_score': 0.8571428571428572, 'matthews_corrcoef': 0.541109329024367, 'cohen_kappa': 0.5322819122720552, 'true_positive_rate': 0.8421052631578947, 'true_negative_rate': 0.75, 'positive_predictive_value': 0.9230769230769231, 'negative_predictive_value': 0.5714285714285714, 'false_positive_rate': 0.25, 'false_negative_rate': 0.15789473684210525, 'false_discovery_rate': 0.07692307692307693}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-35


 Experiment(HYBIM-3)
332/332 [==============================] - 2s 5ms/sample

fpr : 0.0 , tpr : 0.896551724137931 , gmeans : 0.9468641529479986
optimal_threshold :  0.7 

638/638 [==============================] - 3s 4ms/sample
{'accuracy': 0.863013698630137, 'precision': 0.9607843137254902, 'recall': 0.8596491228070176, 'f1_score': 0.9074074074074074, 'f2_score': 0.8781362007168458, 'matthews_corrcoef': 0.6623399563056614, 'cohen_kappa': 0.6473429951690821, 'true_positive_rate': 0.8596491228070176, 'true_negative_rate': 0.875, 'positive_predictive_value': 0.9607843137254902, 'negative_predictive_value': 0.6363636363636364, 'false_positive_rate': 0.125, 'false_negative_rate': 0.14035087719298245, 'false_discovery_rate': 0.0392156862745098}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


...

https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-65


Invalid metric value: nan for channel positive_predictive_value. Metrics with nan or +/-inf values will not be sent to server
Invalid metric value: nan for channel false_discovery_rate. Metrics with nan or +/-inf values will not be sent to server




 Experiment(HYBIM-33)
332/332 [==============================] - 3s 8ms/sample

fpr : 0.0 , tpr : 0.8275862068965517 , gmeans : 0.909717652294684
optimal_threshold :  0.7777777777777778 

638/638 [==============================] - 5s 7ms/sample
{'accuracy': 0.8082191780821918, 'precision': 0.9387755102040817, 'recall': 0.8070175438596491, 'f1_score': 0.8679245283018868, 'f2_score': 0.8303249097472923, 'matthews_corrcoef': 0.5455660889219097, 'cohen_kappa': 0.525092936802974, 'true_positive_rate': 0.8070175438596491, 'true_negative_rate': 0.8125, 'positive_predictive_value': 0.9387755102040817, 'negative_predictive_value': 0.5416666666666666, 'false_positive_rate': 0.1875, 'false_negative_rate': 0.19298245614035087, 'false_discovery_rate': 0.061224489795918366}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-66
